In [2]:
import pandas as pd 
import numpy as np 


In [3]:
df = pd.read_csv("C:\\Users\\Rober\\Downloads\\Database SQL\\Cotizaciones_historicas_SP_500.csv")

df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Symbol,Company,timestamp,id,Adj Close
0,2000-01-03 00:00:00-05:00,20.168121,20.259973,19.748225,19.813833,2599386.0,0.0,0.0,MMM,3M,2024-10-31 00:33:30,95395fe9-18dc-48a0-b6e8-26bd98c9890c,NaN
1,2000-01-04 00:00:00-05:00,19.498910,19.905685,19.026529,19.026529,3245705.0,0.0,0.0,MMM,3M,2024-10-31 00:33:30,de2f669a-ca8d-4bb6-ad69-ef93bfa287c6,NaN
2,2000-01-05 00:00:00-05:00,19.131507,20.207488,19.131507,19.577646,4424482.0,0.0,0.0,MMM,3M,2024-10-31 00:33:30,fdd7e4f3-23a2-4bdf-9c5c-9c38bb8f733e,NaN
3,2000-01-06 00:00:00-05:00,19.800715,21.519663,19.800715,21.152254,7147057.0,0.0,0.0,MMM,3M,2024-10-31 00:33:30,e1497e88-87da-4811-bb11-b1b195efcc83,NaN
4,2000-01-07 00:00:00-05:00,21.230982,21.795218,20.981670,21.572147,4905035.0,0.0,0.0,MMM,3M,2024-10-31 00:33:30,7526b720-764d-4907-9138-80a88bf3e84e,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2756954,2024-10-24 00:00:00-04:00,187.559998,188.250000,180.059998,181.500000,4485900.0,0.0,0.0,ZTS,Zoetis,2024-10-31 00:33:30,bbad2bbe-5839-4555-8069-2a641f44e5e8,NaN
2756955,2024-10-25 00:00:00-04:00,181.490005,182.029999,179.669998,180.009995,2622900.0,0.0,0.0,ZTS,Zoetis,2024-10-31 00:33:30,f6a801bf-1170-4fca-85a8-c197b8c4bb9e,NaN
2756956,2024-10-28 00:00:00-04:00,181.529999,183.100006,180.699997,182.759995,1909700.0,0.0,0.0,ZTS,Zoetis,2024-10-31 00:33:30,52285b90-b2ae-42aa-8690-730ebd6ab3ee,NaN
2756957,2024-10-29 00:00:00-04:00,182.009995,184.410004,181.039993,181.270004,1696000.0,0.0,0.0,ZTS,Zoetis,2024-10-31 00:33:30,55247e7c-99ba-4763-b984-eaa6006f2f3a,NaN


In [12]:
Symbol = "MMM"
fecha_inicio = '2020-01-01'
fecha_fin ='2021-01-01'

df_empresa = df[df['Symbol'] == Symbol]

df_empresa_periodo = df[(df['Symbol'] == Symbol) &
                        (df['timestamp'] >= fecha_inicio) &
                        (df['timestamp'] <= fecha_fin)]

df_empresa_periodo = df_empresa.sort_values(by='timestamp')

#Calcular rendimiento diario#
df_empresa_periodo['rendimiento'] = df_empresa_periodo['Close'].pct_change()

#Calcular volatilidad diaria (desviacion estandar de los rendimientos diarios)#
volatilidad_diaria = df_empresa_periodo['rendimiento'].std()

#Calcular volatilidad mensual
df_empresa_periodo['mes'] = df_empresa_periodo['timestamp'].dt.to_period('M')
volatilidad_mensual = df_empresa_periodo.groupby('mes')['rendimiento'].std().mean()

print(f"Volatilidad diaria: {volatilidad_diaria}")
print(f"Volatilidad mensual promedio: {volatilidad_mensual}")

Volatilidad diaria: 0.08376859734198079
Volatilidad mensual promedio: 0.08376859734198083
